<a href="https://colab.research.google.com/github/matari18/beltel_test/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Подключение библиотек и загрузка данных


In [381]:
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt

In [382]:
sales = pd.read_csv(
    "https://raw.githubusercontent.com/matari18/beltel_test/main/sales_raw.csv")

In [383]:
discounts = pd.read_csv(
    "https://raw.githubusercontent.com/matari18/beltel_test/main/discounts_raw.csv")

#Обзор данных


In [384]:
sales.head()

,date,item_id,qnty
0,2017-01-01,100001,7.0
1,2017-01-01,100003,2.0
2,2017-01-01,100006,13.0
3,2017-01-01,100010,5.0
4,2017-01-01,100035,53.0


In [385]:
sales.shape

(147143, 3)

In [386]:
discounts.head()

,item_id,promo_typeCode,sale_price_before_promo,sale_price_time_promo,date_start,date_end
0,100551,5.0,159.9,139.9,2023-03-02,2023-03-15
1,100553,5.0,159.9,139.9,2023-03-02,2023-03-15
2,100554,5.0,159.9,139.9,2023-03-02,2023-03-15
3,100558,5.0,159.9,139.9,2023-03-02,2023-03-15
4,100530,2.0,36.9,24.6,2021-09-09,2021-09-22


In [387]:
discounts.shape

(2081, 6)


#Предобработка

In [388]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147143 entries, 0 to 147142
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   date     147143 non-null  object 
 1   item_id  147143 non-null  int64  
 2   qnty     147143 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.4+ MB


In [380]:
sales.describe()

,item_id,qnty
count,147143.000000,147143.000000
mean,99552.401038,8.778191
std,8819.498659,27.696091
min,11200.000000,-3.000000
25%,100257.000000,1.000000
50%,100486.000000,3.000000
75%,100601.000000,7.000000
max,100691.000000,1201.000000


from matplotlib import pyplot as plt
_df_9['item_id'].plot(kind='hist', bins=20, title='item_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_10['qnty'].plot(kind='hist', bins=20, title='qnty')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_11.plot(kind='scatter', x='item_id', y='qnty', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_12['item_id'].plot(kind='line', figsize=(8, 4), title='item_id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['qnty'].plot(kind='line', figsize=(8, 4), title='qnty')
plt.gca().spines[['top', 'right']].set_visible(False)

Переведем *date* в формат datetime

In [389]:
sales['date'] = pd.to_datetime(sales['date'], format='%Y-%m-%d').dt.date

In [390]:
discounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2081 entries, 0 to 2080
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   item_id                  2081 non-null   int64  
 1   promo_typeCode           2041 non-null   float64
 2   sale_price_before_promo  2081 non-null   float64
 3   sale_price_time_promo    2081 non-null   float64
 4   date_start               2081 non-null   object 
 5   date_end                 2081 non-null   object 
dtypes: float64(3), int64(1), object(2)
memory usage: 97.7+ KB


Переведем *date_start* и *date_end* в формат datetime.

In [391]:
discounts['date_start'] = pd.to_datetime(discounts['date_start'], format='%Y-%m-%d').dt.date
discounts['date_end'] = pd.to_datetime(discounts['date_end'], format='%Y-%m-%d').dt.date

В столбце *promo_typeCode* есть пропуски.
Их всего 1.92 %, поэтому можем удалить эти строки.

In [392]:
round(discounts['promo_typeCode'].isnull().sum() * 100 / len(discounts), 2)

1.92

In [393]:
discounts = discounts.dropna()
discounts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2041 entries, 0 to 2080
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   item_id                  2041 non-null   int64  
 1   promo_typeCode           2041 non-null   float64
 2   sale_price_before_promo  2041 non-null   float64
 3   sale_price_time_promo    2041 non-null   float64
 4   date_start               2041 non-null   object 
 5   date_end                 2041 non-null   object 
dtypes: float64(3), int64(1), object(2)
memory usage: 111.6+ KB


#Собираем финальный датафрейм

Отберем товары из *sales*, которые участвовали в продажах в декабре 2023 и объединим с таблицей *discounts*.

In [394]:
date_start = pd.to_datetime('2023-12-01', format='%Y-%m-%d').date()
date_end = pd.to_datetime('2023-12-31', format='%Y-%m-%d').date()
df = sales[(sales['date'] >= date_start) & (sales['date'] <= date_end)]

In [395]:
df = df.merge(discounts, how='left', on='item_id')

Оставляем только те объекты, в которых дата покупки находится в рамках дат акции.

In [396]:
df = df[((df['date'] >= df['date_start']) & (df['date'] <= df['date_end'])) | df['promo_typeCode'].isna()]

In [397]:
df.isna().sum(axis=0)

date                         0
item_id                      0
qnty                         0
promo_typeCode             355
sale_price_before_promo    355
sale_price_time_promo      355
date_start                 355
date_end                   355
dtype: int64

In [398]:
df.sample(5)

,date,item_id,qnty,promo_typeCode,sale_price_before_promo,sale_price_time_promo,date_start,date_end
12520,2023-12-10,100188,1.0,NaN,NaN,NaN,NaN,NaN
34826,2023-12-28,100165,37.0,5.0,63.9,54.9,2023-12-26,2024-01-10
3580,2023-12-03,100348,1.0,6.0,299.9,229.9,2023-11-23,2023-12-06
22904,2023-12-19,100001,4.0,NaN,NaN,NaN,NaN,NaN
20665,2023-12-17,100375,1.0,6.0,279.9,179.9,2023-12-07,2023-12-20


Видим, что есть товары, для которых не нашлось акций. Нужно правильно отразить это в датафрейме. Заполним 0 пропуски в *promo_typeCode*.

In [399]:
df['promo_typeCode'] = df['promo_typeCode'].fillna(0)

<ipython-input-399-f581440d921c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['promo_typeCode'] = df['promo_typeCode'].fillna(0)


Добавим столбец, отражающий процент цены по акции по отношению к цене до акции. Для товаров без акций заполним 100.

In [400]:
df['promo_discount'] = round(100 * df['sale_price_time_promo']/df['sale_price_before_promo'], 2)
df['promo_discount'] = df['promo_discount'].fillna(100.)

<ipython-input-400-0d5619d65d8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['promo_discount'] = round(100 * df['sale_price_time_promo']/df['sale_price_before_promo'], 2)
<ipython-input-400-0d5619d65d8d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['promo_discount'] = df['promo_discount'].fillna(100.)


In [401]:
sorted(df['promo_discount'].unique())

[60.99,
 64.27,
 64.28,
 64.49,
 66.64,
 70.57,
 70.82,
 71.39,
 71.41,
 72.4,
 72.7,
 73.9,
 74.25,
 74.97,
 76.66,
 78.09,
 78.25,
 78.54,
 78.72,
 78.92,
 79.97,
 79.98,
 82.98,
 83.31,
 85.92,
 92.0,
 93.33,
 100.0,
 102.66]

In [402]:
df[df['promo_discount'] > 100].groupby(['item_id', 'promo_typeCode']).count()

,,date,qnty,sale_price_before_promo,sale_price_time_promo,date_start,date_end,promo_discount
item_id,promo_typeCode,,,,,,,
100354,5.0,10,10,10,10,10,10,10


Как видим, есть 10 записей с товаром *item_id*=100354 и
*promo_typeCode*=5.0, для которого цена во время акции составляет 102,66%. На первый взгляд выглядит как ошибка, можно удалить такие строки.

 Удалим столбцы *sale_price_before_promo*, *sale_price_time_promo*, *date_start* и *date_end*.

In [403]:
df = df.drop(['sale_price_before_promo', 'sale_price_time_promo', 'date_start', 'date_end'], axis=1)

Добавим столбец *day_of_week*.

In [404]:
df['day_of_week']= pd.to_datetime(df['date']).dt.dayofweek
df.head(5)

,date,item_id,qnty,promo_typeCode,promo_discount,day_of_week
11,2023-12-01,100545,3.0,44.0,78.25,4
162,2023-12-01,100006,3.0,0.0,100.00,4
166,2023-12-01,100601,9.0,6.0,74.97,4
203,2023-12-01,100035,7.0,0.0,100.00,4
265,2023-12-01,100604,14.0,6.0,74.97,4


Проведем кодирование категориального признака *promo_typeCode*

In [405]:
df = pd.get_dummies(df, columns=['promo_typeCode'], drop_first=True)
df.head()

,date,item_id,qnty,promo_discount,day_of_week,promo_typeCode_5.0,promo_typeCode_6.0,promo_typeCode_10.0,promo_typeCode_44.0
11,2023-12-01,100545,3.0,78.25,4,False,False,False,True
162,2023-12-01,100006,3.0,100.00,4,False,False,False,False
166,2023-12-01,100601,9.0,74.97,4,False,True,False,False
203,2023-12-01,100035,7.0,100.00,4,False,False,False,False
265,2023-12-01,100604,14.0,74.97,4,False,True,False,False
